# Spotipy Imports

In [1]:
import os
import json
import spotipy
import spotipy.util as util
from json.decoder import JSONDecodeError

from config import *

In [2]:
import numpy as np
import pandas as pd

### Username number

In [3]:
username = '12166626141'

In [4]:
scope_list = ['user-read-currently-playing','user-read-playback-state',\
              'user-follow-read','user-library-read','user-top-read','user-read-recently-played']

# Throw all permissions from the list into a string for token function:
scope = ' '.join(scope_list)

## Authourize access

In [5]:
token = util.prompt_for_user_token(username,scope=scope,\
    client_id=client_id,\
    client_secret=client_secret,\
    redirect_uri=redirect_uri)

# except:
#     print("exception")
#     os.remove(f'.cache-{username}')
#     token = util.prompt_for_user_token(username,scope=scope,\
#         client_id=client_id,\
#         client_secret=client_secret,\
#         redirect_uri=redirect_uri)

In [6]:
# sp = spotipy.Spotify(auth=token)
sp = spotipy.Spotify(auth=token)

#### To see what is stored: print(json.dumps(VARIABLE, sort_keys=True, indent=4))

### Test if search Artist is working!

In [11]:
search_input = input("Artist to search for: ")
search_results = sp.search(
	search_input,
	limit=1,
	offset=0,
	type="artist"
	)
artist = search_results['artists']['items'][0]
print(json.dumps(search_results, sort_keys=True, indent=4))

Artist to search for: Queen
{
    "artists": {
        "href": "https://api.spotify.com/v1/search?query=Queen&type=artist&market=US&offset=0&limit=1",
        "items": [
            {
                "external_urls": {
                    "spotify": "https://open.spotify.com/artist/1dfeR4HaWDbWqFHLkxsg1d"
                },
                "followers": {
                    "href": null,
                    "total": 11142804
                },
                "genres": [
                    "glam rock",
                    "rock"
                ],
                "href": "https://api.spotify.com/v1/artists/1dfeR4HaWDbWqFHLkxsg1d",
                "id": "1dfeR4HaWDbWqFHLkxsg1d",
                "images": [
                    {
                        "height": 806,
                        "url": "https://i.scdn.co/image/b040846ceba13c3e9c125d68389491094e7f2982",
                        "width": 999
                    },
                    {
                        "height": 516,
   

In [12]:
search_results['artists']['items'] == []

False

In [14]:
artist_id = artist["id"]

## Pull entire discography of albums

#### Note: artist_id defined in search section above

<hr>

### Create a track list of all unique songs in discography

In [15]:
# Use artist_id from above.
albums = sp.artist_albums(artist_id=artist_id, album_type="album")
album_ids = [album["id"] for album in albums["items"]]
album_names = [album["name"] for album in albums["items"]]
album_total_tracks = [album["total_tracks"] for album in albums["items"]]

In [16]:
album_tracks = [sp.album_tracks(album_id) for album_id in album_ids]
print(json.dumps(album_tracks, sort_keys=True, indent=4))

[
    {
        "href": "https://api.spotify.com/v1/albums/3BHe7LbW5yRjyqXNJ3A6mW/tracks?offset=0&limit=50",
        "items": [
            {
                "artists": [
                    {
                        "external_urls": {
                            "spotify": "https://open.spotify.com/artist/1dfeR4HaWDbWqFHLkxsg1d"
                        },
                        "href": "https://api.spotify.com/v1/artists/1dfeR4HaWDbWqFHLkxsg1d",
                        "id": "1dfeR4HaWDbWqFHLkxsg1d",
                        "name": "Queen",
                        "type": "artist",
                        "uri": "spotify:artist:1dfeR4HaWDbWqFHLkxsg1d"
                    }
                ],
                "available_markets": [
                    "AD",
                    "AE",
                    "AR",
                    "AT",
                    "AU",
                    "BE",
                    "BG",
                    "BH",
                    "BO",
                    "BR"

In [17]:
track_ids = []
track_names = []
track_numbers = []

for disc in album_tracks:
    for song in disc["items"]:
        # Now we can pull song_id, song_name, and song_track_number
        track_ids.append(song["id"])
        track_names.append(song["name"])
        track_numbers.append(song["track_number"])
        
print(track_names)

['20th Century Fox Fanfare', 'Somebody To Love - 2011 Remaster', 'Doing All Right - …revisited', 'Keep Yourself Alive - Live at The Rainbow', 'Killer Queen - 2011 Remaster', 'Fat Bottomed Girls - Live in Paris', 'Bohemian Rhapsody - 2011 Remaster', "Now I'm Here - Live at The Hammersmith Odeon", 'Crazy Little Thing Called Love - 2011 Remaster', 'Love Of My Life - Live at Rock in Rio Festival', 'We Will Rock You - Movie Mix', 'Another One Bites The Dust - 2011 Remaster', 'I Want To Break Free', 'Under Pressure - Remastered', 'Who Wants To Live Forever - 2011 Remaster', 'Bohemian Rhapsody - Live Aid', 'Radio Ga Ga - Live Aid', 'Ay-Oh - Live Aid', 'Hammer To Fall - Live Aid', 'We Are The Champions - Live Aid', "Don't Stop Me Now - …revisited", 'The Show Must Go On - 2011 Remaster', 'My Fairy King - BBC Session / February 5th 1973, Langham 1 Studio', 'Keep Yourself Alive - BBC Session / February 5th 1973, Langham 1 Studio', 'Doing All Right - BBC Session / February 5th 1973, Langham 1 Stud

In [18]:
track_features = sp.audio_features(track_ids)
print(json.dumps(track_features, sort_keys=True, indent=4))
print(type(track_features))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
df_feat = pd.DataFrame(track_features)
df_feat['name'] = track_names
df_feat['track_number'] = track_numbers

df_feat.drop(columns=['analysis_url','track_href','type','uri'], inplace=True)
df_feat.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df_stats = df_feat.drop(columns=['id','key','mode','time_signature', 'duration_ms', 'loudness','tempo','track_number', 'name'])

In [ ]:
df_stats

In [ ]:
df_stats.mean()
df_stats.columns
pd.melt(df_stats)

In [ ]:
sns.set(style='whitegrid')
sns.set(font_scale=1.3) 

fig = plt.figure(figsize=(15,5))
ax = fig.add_axes([.1,.1,.8,.8])
ax.set(ylim = (0,1))
sns.barplot(x=df_stats.columns, y=df_stats.mean())

ax.set_ylabel('Top Song Average', fontsize=14)
plt.show()

In [ ]:
sns.set(style='whitegrid')
sns.set(font_scale=1.3) 

fig = plt.figure(figsize=(15,5))
ax = fig.add_axes([.1,.1,.8,.8])
ax.set(ylim = (0,1))
sns.boxplot(x='variable', y='value', data=pd.melt(df_stats))
ax.set_xlabel('')
ax.set_ylabel('Top Song Average', fontsize=14)
plt.show()

## Lets make charts of the following parameters:
 - Key Signature (key map needed)
 - Tempo(make bins)
 - Mode (major=1 or minor=0)
 - Duration (make bins)
 - loudness (dB)
 - time signature (not sure if I trust it)
 <hr>

In [ ]:
# map key numbers with actual letter Keys, and mode with major/minor
keyMap = {
    0:"C",
    1:"C#/Db",
    2:"D",
    3:"D#/Eb",
    4:"E",
    5:"F",
    6:"F#/Gb",
    7:"G",
    8:"G#/Ab",
    9:"A",
    10:"A#/Bb",
    11:"B",
}

modeMap = {
    0:"minor",
    1:"major",
}

In [ ]:
df_feat["key"] = df_feat["key"].map(keyMap)
df_feat["mode"] = df_feat["mode"].map(modeMap)

In [ ]:
df_feat

## Key Signature

In [ ]:
keyGroup = df_feat.groupby(by="key").count()

keyGroup.reset_index(inplace=True)

In [ ]:
sns.set(font_scale=1.3) 

fig = plt.figure(figsize=(15,5))
ax = fig.add_axes([.1,.1,.8,.8])
sns.barplot(x="key",y="id",data=keyGroup)
ax.set_title("Key Signature", fontsize=18)
ax.set_ylabel('Song Count', fontsize=14)
plt.show()

## Mode

In [ ]:
modeGroup = df_feat.groupby(by="mode").count()

modeGroup.reset_index(inplace=True)

In [ ]:
sns.set(font_scale=1.3) 

fig = plt.figure(figsize=(15,5))
ax = fig.add_axes([.1,.1,.8,.8])
sns.barplot(x="mode",y="id",data=modeGroup)
ax.set_title("Mode Chart", fontsize=18)
ax.set_ylabel('Song Count', fontsize=14)
plt.show()

## Duration

In [ ]:
import datetime
# convert duration to minutes
df_feat["duration"] = pd.to_timedelta(df_feat["duration_ms"], unit="ms")
# df_feat["duration"] / pd.Timedelta(minutes=1)

In [ ]:
fig = plt.figure(figsize=(15,5))
ax = fig.add_axes([.1,.1,.8,.8])
sns.distplot(df_feat["duration"] / pd.Timedelta(minutes=1),
                bins=20)
ax.set_xlabel("Minutes")
ax.set_ylabel("Frequency of Song Duration")
ax.set_title("Top Songs Duration")


In [ ]:
df_feat["tempo"].head()

## Tempo

In [ ]:
fig = plt.figure(figsize=(15,5))
ax = fig.add_axes([.1,.1,.8,.8])
sns.distplot(df_feat["tempo"],
                bins=20)
ax.set_xlabel("Tempo")
ax.set_ylabel("Frequency of Tempo")
ax.set_title("Top Songs Duration")

## Loudness

In [ ]:
fig = plt.figure(figsize=(15,5))
ax = fig.add_axes([.1,.1,.8,.8])
sns.distplot(df_feat["loudness"],
                bins=20)
ax.set_xlabel("Loudness")
ax.set_ylabel("Frequency of Tempo")
ax.set_title("Top Songs Duration")

<hr>

## Test out the recommender function:

In [ ]:
sp.recommendation_genre_seeds()

In [ ]:
sp.recommendations(seed_genres=["alternative"], target_energy=.8, min_danceability=.6)

<hr>

# Refresh Token

In [ ]:
import spotipy.oauth2 as oauth

In [ ]:
spo = oauth.SpotifyOAuth(client_id=client_id,\
                         client_secret=client_secret,\
                         redirect_uri=redirect_uri,\
                         scope=scope,\
                         cache_path=f'.cache-{username}'
                        )

In [ ]:
cache_token_info = spo.get_cached_token()

In [ ]:
refreshed_token = cache_token_info["refresh_token"]

In [ ]:
new_token = spo.refresh_access_token(refreshed_token)

In [ ]:
new_token = new_token["access_token"]

#### When token expires, need to run through this refresh token process!